In [ ]:
%matplotlib inline


# Variational with birth and merge proposals for DP mixtures of Gaussians

How to train a DP mixture model.

We'll show that despite diverse, poor quality initializations, our proposal moves that insert new clusters (birth) and remove redundant clusters (merge) can consistently recover the same ideal posterior with 8 clusters.


Read dataset from file.



In [1]:
import bnpy
import numpy as np
import os
import torch
from bnpy.data.XData import XData
# from sklearn.manifold import TSNE
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import scipy.io as scio
from torchvision.datasets import MNIST, CIFAR10, FashionMNIST, SVHN, KMNIST, STL10
from scipy.optimize import linear_sum_assignment

# from matplotlib import pylab
# import seaborn as sns

In [31]:
seed = 1
np.random.seed(seed)
# n_components = 3
num_digits = 10
dataset_type = 'mnist'
dataset_path = 'dataset/mnist/'
# dataset_type = 'fashion-mnist'
# dataset_path = 'dataset/fashion/'
# dataset_type = 'reuters10k'
# dataset_path = 'dataset/reuters10k/'
# dataset_type = 'har'
# dataset_path = 'dataset/har/'


# Define the transforms to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

class Reuters10kDataset(Dataset):
    '''Reuters 10k dataset torchvision wrapper
        Args:
            data_path: path/to/save/the/reuters10k.mat (filename not included)
            train: if True split 80% of total dataset as train dataset otherwise 20% as test set
        Outputs:
            data: shape (batch_size, 2000) text tokens
            labels: shape (batch_size, 1) indicators of text categories
    '''
    def __init__(self, data_path, train=True):
        # load data and labels from data_path
        self.content=scio.loadmat(data_path+'reuters10k.mat')
        length = len(self.content['X'])
        train_limits = int(0.8*length)
        self.labels = self.content['Y'].squeeze()

        if train is True: # split train/test == 8/2
            self.data = self.content['X'][:train_limits]
            self.labels = self.labels[:train_limits]
        else:
            self.data = self.content['X'][train_limits:]
            self.labels = self.labels[train_limits:]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # get data and label at idx
        x = self.data[idx]
        y = self.labels[idx]
        # convert data to tensor and normalize
        x = torch.tensor(x, dtype=torch.float32)
        # return data and label as tuple
        return x, y

class HARDataset(Dataset):
    '''HAR dataset torchvision wrapper
        Args:
            data_path: path/to/save/the/HAR.mat(filename not included), data from VaDE
            train: if True split 80% of total dataset as train dataset otherwise 20% as test set
        Outputs:
            data: shape (batch_size, 561) sensor
            labels: shape (batch_size, 1) indicators of action categories
    '''
    def __init__(self, data_path, train=True):
        # load data and labels from data_path
        self.content=scio.loadmat(data_path+'HAR.mat')
        self.labels = self.content['Y'] -1
        self.labels = self.labels.squeeze()
        self.data = self.content['X']
        # print(self.labels.shape)
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # get data and label at idx
        x = self.data[idx]
        y = self.labels[idx]
        # convert data to tensor and normalize
        x = torch.tensor(x, dtype=torch.float32)
        # return data and label as tuple
        return x, y


def configure_subset(dataset:Dataset, num_digits:int):
    '''
    limit the representations (type of digits) in the dataset, to build a subset 
    e.g. num_digits = 3, the subset should only contents digits [0,1,2]
    '''
    if num_digits < 4:
        digits = list(np.arange(num_digits))
        select_idxs = [i for i in range(len(dataset)) if dataset.labels[i] in digits]
        subset = Subset(dataset, select_idxs)
    else:
        subset = dataset
    return subset

def train_dataset():

    if dataset_type == 'mnist':
        full_train_dataset = MNIST(root = dataset_path,
                                    train = True, 
                                    transform=transform, 
                                    download=True)
    elif dataset_type == 'fashion-mnist':
        full_train_dataset = FashionMNIST(root = dataset_path,
                            train = True, 
                            transform = transform, 
                            download = True,
                            )
    elif dataset_type == 'reuters10k':
        full_train_dataset = Reuters10kDataset(data_path=dataset_path, train=True)
    else:
        full_train_dataset = HARDataset(data_path=dataset_path)

    
    dataset = configure_subset(full_train_dataset, num_digits)
    labels = np.array([y for x, y in dataset])

    if dataset_type != 'reuters10k':
        data = np.array([np.array(x.flatten()) for x, y in dataset])
    else:
        data = np.array([np.array(x) for x, y in dataset])
    
    return data, labels

def test_dataset():
    if dataset_type == 'mnist':
        full_test_dataset = MNIST(root = dataset_path,
                                  train = False,
                                  transform=transform, 
                                  download=True)
    elif dataset_type == 'fashion-mnist':
        full_test_dataset = FashionMNIST(root = dataset_path,
                            train = True, 
                            transform = transform, 
                            download = True,
                            )
    elif dataset_type == 'reuters10k':
        full_test_dataset = Reuters10kDataset(data_path=dataset_path, train=False)
    else:
        full_test_dataset = HARDataset(data_path=dataset_path)
    
    dataset = configure_subset(full_test_dataset, num_digits)
    if dataset_type != 'reuters10k':
        data = np.array([np.array(x.flatten()) for x, y in dataset])
    else:
        data = np.array([np.array(x) for x, y in dataset])
    labels = np.array([y for x, y in dataset])
    return data, labels

def unsupervised_clustering_accuracy(y: np.ndarray, y_pred: np.ndarray) -> tuple:
    """Unsupervised Clustering Accuracy
    """
    assert len(y_pred) == len(y)
    u = np.unique(y)
    n_true_clusters = len(u)
    v = np.unique(y_pred)
    n_pred_clusters = len(v)
    map_u = dict(zip(u, range(n_true_clusters)))
    map_v = dict(zip(v, range(n_pred_clusters)))
    # inv_map_u = {v: k for k, v in map_u.items()}
    # inv_map_v = {v: k for k, v in map_v.items()}
    r = np.zeros((n_pred_clusters, n_true_clusters), dtype=np.int64)
    for y_pred_, y_ in zip(y_pred, y):
        if y_ in map_u:
            r[map_v[y_pred_], map_u[y_]] += 1
    reward_matrix  = np.concatenate((r, r, r), axis=1)
    cost_matrix = reward_matrix.max() - reward_matrix
    row_assign, col_assign = linear_sum_assignment(cost_matrix)

    # Construct optimal assignments matrix
    row_assign = row_assign.reshape((-1, 1))  # (n,) to (n, 1) reshape
    col_assign = col_assign.reshape((-1, 1))  # (n,) to (n, 1) reshape
    # assignments = np.concatenate((row_assign, col_assign), axis=1)
    # assignments = [[inv_map_v[x], inv_map_u[y%n_true_clusters]] for x, y in assignments]

    optimal_reward = reward_matrix[row_assign, col_assign].sum() * 1.0
    return optimal_reward / y_pred.size

train_data, train_label = train_dataset()
test_data, test_label = test_dataset()
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [33]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100, random_state = 1)

reduced_train_data = pca.fit_transform(train_data)
reduced_test_data = pca.fit_transform(test_data)
print(reduced_train_data.shape)
print(reduced_test_data.shape)


(60000, 100)
(10000, 100)


In [34]:

import os
save_dir = "dpmm_save/"

train_data_for_dpmm = XData(reduced_train_data)
test_data_for_dpmm = XData(reduced_test_data)

model, info_dict = bnpy.run(train_data_for_dpmm, 'DPMixtureModel', 'DiagGauss', 'memoVB',
                            output_path=save_dir,
                            initname='randexamples',
                            K=4, gamma0=5.0,
                            ####################################################################
                            sF=0.1, ECovMat='eye', # sF=0.001 is able to classify all clusters
                            ####################################################################
                            moves='birth,merge,shuffle', nBatch=1, nLap=100,
                            b_startLap=1,

                            # birth kwargs
                            b_stopLap=100,
                            b_Kfresh=5,
                            # b_minNumAtomsForNewComp=16.0,
                            # b_minNumAtomsForTargetComp=16.0,
                            # b_minNumAtomsForRetainComp=16.0,
                            # b_minPercChangeInNumAtomsToReactivate=0.02,
                            # b_debugWriteHTML=0,
                            
                            # merge kwargs
                            m_startLap=2,
                            m_maxNumPairsContainingComp=50,
                            m_nLapToReactivate=1,
                            m_pair_ranking_procedure='obsmodel_elbo',
                            m_pair_ranking_direction='descending',
                            )

Dataset Summary:
X Data
  total size: 60000 units
  batch size: 60000 units
  num. batches: 1
Allocation Model:  DP mixture with K=0. Concentration gamma0= 5.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 102  ...
  beta = [ 10  10]  ...
  Expectations
  E[  mean[k]] = 
  [ 0  0] ...
  E[ covar[k]] = 
  [[0.1 0. ]
   [0.  0.1]] ...
Initialization:
  initname = randexamples
  K = 4 (number of clusters)
  seed = 1607680
  elapsed_time: 0.2 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: dpmm_save/1
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
BIRTH @ lap 1.00 : Added 14 states. 4/4 succeeded. 0/4 failed eval phase. 0/4 failed build phase.
    1.000/100 after      6 sec. |    957.9 MiB | K   18 | loss  1.814781628e+00 |  
BIRTH @ lap 2.000 : None attempted. 0 past failures. 0

In [30]:
test_preds =  []
train_preds = []

LP_test = model.calc_local_params(test_data_for_dpmm)
resp_test = LP_test['resp'] 
preds_eval = resp_test.argmax(axis=1)
test_preds = test_preds + preds_eval.tolist()
test_preds = np.array(test_preds)

LP_train = model.calc_local_params(train_data_for_dpmm)
resp_train = LP_train['resp'] 
preds_t = resp_train.argmax(axis=1)
train_preds = train_preds + preds_t.tolist()
train_preds = np.array(train_preds)


print('test accuracy: ', unsupervised_clustering_accuracy(test_label, test_preds)*100)
print('train accuracy: ', unsupervised_clustering_accuracy(train_label, train_preds)*100)

test accuracy:  48.3
train accuracy:  58.6125
